In [1]:
import jax
import jax.numpy as jnp
import jax.tree_util as jtu

import genjax
from genjax import normal
from genjax import tfp_uniform
from genjax import trace
from genjax import gen
from genjax.inference import smc
from genjax import index_choice_map, choice_map
from genjax import diff, NoChange, UnknownChange
from genjax import index_select

console = genjax.pretty()
key = jax.random.PRNGKey(314159)

In [2]:
@gen(genjax.Unfold, max_length=10)
def chain(z_prev):
    z = normal(z_prev, 1.0) @ "z"
    x = normal(z, 1.0) @ "x"
    return z


key, tr = chain.simulate(key, (5, 0.0))
tr

VectorTrace
├── gen_fn
│   └── UnfoldCombinator
│       ├── max_length
│       │   └── (const) 10
│       └── kernel
│           └── BuiltinGenerativeFunction
│               └── source
│                   └── <function chain>
├── inner
│   └── BuiltinTrace
│       ├── gen_fn
│       │   └── BuiltinGenerativeFunction
│       │       └── source
│       │           └── <function chain>
│       ├── args
│       │   └── tuple
│       │       └──  f32[10]
│       ├── retval
│       │   └──  f32[10]
│       ├── choices
│       │   └── Trie
│       │       ├── :z
│       │       │   └── DistributionTrace
│       │       │       ├── gen_fn
│       │       │       │   └── Normal
│       │       │       ├── args
│       │       │       │   └── tuple
│       │       │       │       ├──  f32[10]
│       │       │       │       └──  f32[10]
│       │       │       ├── value
│       │       │       │   └──  f32[10]
│       │       │       └── score
│       │       │           └──  f32[10]
│       │       └── :x
│       │           └── DistributionTrace
│       │               ├── gen_fn
│       │               │   └── Normal
│       │               ├── args
│       │               │   └── tuple
│       │               │       ├──  f32[10]
│       │               │       └──  f32[10]
│       │               ├── value
│       │               │   └──  f32[10]
│       │               └── score
│       │                   └──  f32[10]
│       ├── cache
│       │   └── Trie
│       └── score
│           └──  f32[10]
├── args
│   └── tuple
│       ├── (const) 5
│       └── (const) 0.0
├── retval
│   └──  f32[10]
└── score
    └──  f32[]

In [3]:
obs = index_choice_map(
    [0],
    choice_map({"x": jnp.array([1.0])}),
)
obs

IndexChoiceMap
├── indices
│   └──  i32[1]
└── inner
    └── BuiltinChoiceMap
        └── trie
            └── Trie
                └── :x
                    └── ValueChoiceMap
                        └── value
                            └──  f32[1]

In [4]:
key, (_, _, new_tr, _) = chain.update(
    key, tr, obs, (diff(5, UnknownChange), diff(0.0, NoChange))
)

In [5]:
tr

VectorTrace
├── gen_fn
│   └── UnfoldCombinator
│       ├── max_length
│       │   └── (const) 10
│       └── kernel
│           └── BuiltinGenerativeFunction
│               └── source
│                   └── <function chain>
├── inner
│   └── BuiltinTrace
│       ├── gen_fn
│       │   └── BuiltinGenerativeFunction
│       │       └── source
│       │           └── <function chain>
│       ├── args
│       │   └── tuple
│       │       └──  f32[10]
│       ├── retval
│       │   └──  f32[10]
│       ├── choices
│       │   └── Trie
│       │       ├── :z
│       │       │   └── DistributionTrace
│       │       │       ├── gen_fn
│       │       │       │   └── Normal
│       │       │       ├── args
│       │       │       │   └── tuple
│       │       │       │       ├──  f32[10]
│       │       │       │       └──  f32[10]
│       │       │       ├── value
│       │       │       │   └──  f32[10]
│       │       │       └── score
│       │       │           └──  f32[10]
│       │       └── :x
│       │           └── DistributionTrace
│       │               ├── gen_fn
│       │               │   └── Normal
│       │               ├── args
│       │               │   └── tuple
│       │               │       ├──  f32[10]
│       │               │       └──  f32[10]
│       │               ├── value
│       │               │   └──  f32[10]
│       │               └── score
│       │                   └──  f32[10]
│       ├── cache
│       │   └── Trie
│       └── score
│           └──  f32[10]
├── args
│   └── tuple
│       ├── (const) 5
│       └── (const) 0.0
├── retval
│   └──  f32[10]
└── score
    └──  f32[]

In [6]:
new_tr

VectorTrace
├── gen_fn
│   └── UnfoldCombinator
│       ├── max_length
│       │   └── (const) 10
│       └── kernel
│           └── BuiltinGenerativeFunction
│               └── source
│                   └── <function chain>
├── inner
│   └── BuiltinTrace
│       ├── gen_fn
│       │   └── BuiltinGenerativeFunction
│       │       └── source
│       │           └── <function chain>
│       ├── args
│       │   └── tuple
│       │       └──  f32[10]
│       ├── retval
│       │   └──  f32[10]
│       ├── choices
│       │   └── Trie
│       │       ├── :z
│       │       │   └── DistributionTrace
│       │       │       ├── gen_fn
│       │       │       │   └── Normal
│       │       │       ├── args
│       │       │       │   └── tuple
│       │       │       │       ├──  f32[10]
│       │       │       │       └──  f32[10]
│       │       │       ├── value
│       │       │       │   └──  f32[10]
│       │       │       └── score
│       │       │           └──  f32[10]
│       │       └── :x
│       │           └── DistributionTrace
│       │               ├── gen_fn
│       │               │   └── Normal
│       │               ├── args
│       │               │   └── tuple
│       │               │       ├──  f32[10]
│       │               │       └──  f32[10]
│       │               ├── value
│       │               │   └──  f32[10]
│       │               └── score
│       │                   └──  f32[10]
│       ├── cache
│       │   └── Trie
│       └── score
│           └──  f32[10]
├── args
│   └── tuple
│       ├── (const) 5
│       └── (const) 0.0
├── retval
│   └──  f32[10]
└── score
    └──  f32[]